In [91]:
import geopandas as gpd
import pandas as pd
import numpy as np
import folium
from matplotlib import pyplot as plt
from geopy.geocoders import Nominatim
from folium import Marker
import warnings 
warnings.filterwarnings('ignore')

In [92]:
geolocator = Nominatim(user_agent="Guerra")
location = geolocator.geocode("Romania")

In [93]:
point = location.point
print("Latitude:", point.latitude)
print("Longitude:", point.longitude)

Latitude: 45.9852129
Longitude: 24.6859225


In [94]:
countries = pd.read_csv("../Dados/UKR_refugee_by_countries.csv")
#countries=pd.DataFrame(pd.unique(countries["country"]))
countries=countries.groupby('country')[['individuals']].max().reset_index()





In [95]:
def my_geocoder(row):
    try:
        point = geolocator.geocode(row).point
        return pd.Series({'Latitude': point.latitude, 'Longitude': point.longitude})
    except:
        return None

countries[['Latitude', 'Longitude']] = countries.head(8).apply(lambda x: my_geocoder(x['country']), axis=1)

print("{}% of addresses were geocoded!".format(
    (1 - sum(np.isnan(countries["Latitude"])) / len(countries)) * 100))

# Drop universities that were not successfully geocoded
countries = countries.loc[~np.isnan(countries["Latitude"])]
countries = gpd.GeoDataFrame(
    countries, geometry=gpd.points_from_xy(countries.Longitude, countries.Latitude))
countries.crs = {'init': 'epsg:4326'}
display(countries)

87.5% of addresses were geocoded!


,country,individuals,Latitude,Longitude,geometry
0,Belarus,27308,53.425061,27.697136,POINT (27.69714 53.42506)
1,Hungary,1381738,47.181759,19.506094,POINT (19.50609 47.18176)
3,Poland,6114787,52.215933,19.134422,POINT (19.13442 52.21593)
4,Republic of Moldova,606707,47.287961,28.567094,POINT (28.56709 47.28796)
5,Romania,1169027,45.985213,24.685923,POINT (24.68592 45.98521)
6,Russian Federation,2593209,64.686314,97.745306,POINT (97.74531 64.68631)
7,Slovakia,778335,48.741152,19.452865,POINT (19.45286 48.74115)


In [97]:
import branca.colormap as cmp
import requests as rq
import json

# Create a map
m = folium.Map(location=[48, 31],zoom_start=4, tiles='cartodb positron')

# Add points to the map
for idx, row in countries.iterrows():
    Marker([row['Latitude'], row['Longitude']], popup=row['country']).add_to(m)
political_countries_url = (
    "http://geojson.xyz/naturalearth-3.3.0/ne_50m_admin_0_countries.geojson"
)
folium.Choropleth(
    geo_data=political_countries_url,
    data=countries,
    columns=["country", "individuals"],
    key_on="feature.properties.name",
    legend_name = "Refugees Scale",
).add_to(m)

step = cmp.StepColormap(
 ['yellow', 'green', 'purple'],
 vmin=3, vmax=10,
 index=[3, 6, 8, 10],  #for change in the colors, not used fr linear
 caption='Color Scale for Map'    #Caption for Color scale or Legend
)
step

unemployment_dict = countries.set_index('country')['individuals']

usa_step = folium.Map([48, -102], tiles='cartodbpositron', zoom_start=3)
geo_json_data= json.loads(rq.get(political_countries_url).text)
folium.GeoJson(
    geo_json_data,
    style_function=lambda feature: {
        'fillColor': step(unemployment_dict[feature['country']]),
        'color': 'black',       #border color for the color fills
        'weight': 1,            #how thick the border has to be
        'dashArray': '5, 3'  #dashed lines length,space between them
    }
).add_to(usa_step)
step.add_to(usa_step)     #adds colorscale or legend


# Display the map
usa_step

KeyError: 'country'